# Train NF

Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(f"Versión de TensorFlow: {tf.__version__}")

Versión de TensorFlow: 2.15.0


Download data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "..."

In [ ]:
X_train = np.load(path + 'X_train.npy')
y_train = np.load(path + 'y_train.npy') # (alpha, theta, ratio)

We standardize the target variables (parameters) to improve the performance of NN.

In [ ]:
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train)

In [ ]:
y_scaler.mean_

array([1.49225652, 2.07506667, 0.49766667])

In [ ]:
y_scaler.scale_

array([0.90576534, 1.15612128, 0.2267739 ])

Save ``y_scaler``

In [2]:
import joblib
print(f'{joblib.__version__}')

1.4.2


In [ ]:
joblib.dump(y_scaler, path + 'y_scaler.pkl')

Augment the data by rotating the fields by $180$ degrees

In [ ]:
def augment_data(x, y):
    augmented_X = []
    augmented_y = []

    for x_sample, y_sample in zip(x, y):
        augmented_X.append(x_sample)
        augmented_y.append(y_sample)

        # Rotar la matriz en 180 grados
        rotated_x = np.rot90(x_sample,2)
        augmented_X.append(rotated_x)
        augmented_y.append(y_sample)

    return tf.convert_to_tensor(augmented_X), tf.convert_to_tensor(augmented_y)

class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        X_batch = [self.X[i] for i in indexes]
        y_batch = [self.y[i] for i in indexes]

        return augment_data(X_batch, y_batch)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=9, input_shape=(16,16,1), activation='relu'),
    keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu'),
    keras.layers.Conv2D(filters=512, kernel_size=4, activation='relu'),
    keras.layers.Conv2D(filters=1024, kernel_size=1, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(3)
])
model.compile(optimizer=keras.optimizers.AdamW(),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=[keras.metrics.MeanSquaredError()])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 8, 8, 128)         10496     
                                                                 
 conv2d_24 (Conv2D)          (None, 4, 4, 256)         819456    
                                                                 
 conv2d_25 (Conv2D)          (None, 1, 1, 512)         2097664   
                                                                 
 conv2d_26 (Conv2D)          (None, 1, 1, 1024)        525312    
                                                                 
 flatten_6 (Flatten)         (None, 1024)              0         
                                                                 
 dense_16 (Dense)            (None, 300)               307500    
                                                                 
 dense_17 (Dense)            (None, 3)                

In [ ]:
batch_size = 500
generator = DataGenerator(X_train, y_train, batch_size)

In [ ]:
%%time
history = model.fit(generator,
                    epochs = 50,
                    verbose = 1,
                    )

Epoch 1/50
900/900 [==============================] - 51s 52ms/step - loss: 0.4555 - mean_squared_error: 0.4576
Epoch 2/50
900/900 [==============================] - 47s 52ms/step - loss: 0.3387 - mean_squared_error: 0.3156
Epoch 3/50
900/900 [==============================] - 47s 52ms/step - loss: 0.3154 - mean_squared_error: 0.2920
Epoch 4/50
900/900 [==============================] - 47s 52ms/step - loss: 0.3013 - mean_squared_error: 0.2776
Epoch 5/50
900/900 [==============================] - 47s 52ms/step - loss: 0.2920 - mean_squared_error: 0.2678
Epoch 6/50
900/900 [==============================] - 47s 53ms/step - loss: 0.2852 - mean_squared_error: 0.2616
Epoch 7/50
900/900 [==============================] - 48s 53ms/step - loss: 0.2829 - mean_squared_error: 0.2597
Epoch 8/50
900/900 [==============================] - 47s 52ms/step - loss: 0.2772 - mean_squared_error: 0.2540
Epoch 9/50
900/900 [==============================] - 47s 52ms/step - loss: 0.2739 - mean_squared_error:

In [ ]:
X_test = np.load(path + 'X_test.npy')

y_pred_test = y_scaler.inverse_transform(model.predict(X_test, verbose = 0))
y_pred_test

array([[ 2.975435  ,  1.0755346 ,  0.40735844],
       [ 2.991985  ,  1.1226343 ,  0.3593339 ],
       [-0.04057837,  1.0856812 ,  0.29411298],
       ...,
       [ 0.6562753 ,  3.1020145 ,  0.81865036],
       [ 0.6737718 ,  3.0899575 ,  0.7606943 ],
       [ 0.15253878,  2.8015354 ,  0.5997586 ]], dtype=float32)

In [ ]:
np.save(path+'y_pred_NF.npy', y_pred_test)

Save trained NN

In [ ]:
model.save(path+'model.keras')